<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Run your first Python UDF</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p>This notebook can be run on a Free Starter Workspace. To create a Free Starter Workspace navigate to <tt>Start</tt> using the left nav. You can also use your existing Standard or Premium workspace with this Notebook.</p>
    </div>
</div>

<p>This feature is currently in Private Preview. Please reach out support@singlestore.com to confirm if this feature can be enabled in your org.</p>

This Jupyter notebook will help you build your first Cloud Function, showcasing how to leverage the ultra-fast queries of SingleStore to build a responsive API server using FastAPI

## Create some simple tables

This setup establishes a basic relational structure to store some items information.

In [1]:
%%sql
DROP TABLE IF EXISTS items;

CREATE TABLE IF NOT EXISTS
items (
    id INT PRIMARY KEY,
    name VARCHAR(255),
    price FLOAT
);

## Insert sample data

In [2]:
%%sql INSERT into reviews (review_id, store_name, review) values
("1", "Single Pizza", "The staff were very respectful and made thoughtful suggestions. I will definitely go again. 10/10!"),
("2", "Single Pizza", "The food was absolutely amazing and the service was fantastic!"),
("3", "Single Pizza", "The experience was terrible. The food was cold and the waiter was rude."),
("4", "Single Pizza", "I loved the ambiance and the desserts were out of this world!"),
("5", "Single Pizza", "Not worth the price. I expected more based on the reviews");

## Define FastAPI App

Next, we will be defining a FastAPI app that can insert, query and delete data from your table

In [3]:
!pip install nltk

In [4]:
app = FastAPI()

# Get all items
@app.get("/items", response_model=list[Item])
async def get_items():
    def get_items_query():
        result = execute_query("SELECT * FROM items;")
        rows = result.fetchall()
        return [{"id": row[0], "name": row[1], "price": row[2]} for row in rows]

    try:
        return await run_in_thread(get_items_query)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error fetching all items: {str(e)}")

# Insert an item
@app.post("/items", response_model=dict)
async def create_item(item: Item):
    def insert_item_query():
        result = execute_transaction(f"INSERT INTO items (id, name, price) VALUES ({item.id}, '{item.name}', {item.price})")
        return {"message": f"Item with id {item.id} inserted successfully"}

    try:
        return await run_in_thread(insert_item_query)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error while inserting item with id {item.id}: {str(e)}")

# Get item by id
@app.get("/items/{item_id}", response_model=Item)
async def get_item(item_id: int):
    def get_item_query():
        result = execute_query(f"SELECT * FROM items WHERE id={item_id}")
        row = result.fetchone()
        if not row:
            raise HTTPException(status_code=404, detail="Item not found")
        return {"id": row[0], "name": row[1], "price": row[2]}

    try:
        return await run_in_thread(get_item_query)
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error fetching item with id {item_id}: {str(e)}")

# Delete item by id
@app.delete("/items/{item_id}", response_model=dict)
async def delete_item(item_id: int):
    def delete_item_query():
        result = execute_transaction(f"DELETE FROM items WHERE id={item_id}")
        return {"message": f"number of rows deleted: {result.rowcount}"}

    try:
        return await run_in_thread(delete_item_query)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error deleting item with id {item_id}: {str(e)}")

## Start the FastAPI server

The link at which the cloud function will be available interactively will be displayed.

In [5]:
import singlestoredb.apps as apps
connection_info = await apps.run_function_app(app)

## List all registered UDFs

In interactive notebooks, the udf function will be suffixed with `_test` to differentiate it from the published version

In [6]:
%%sql
SHOW functions

## Call the UDF from SQL

You will now be able to run queries like

```
SELECT review_id, store_name, review, review_sentiment_test(review) from reviews order by review_id;
```
from the SQL editor or any other SQL client.

Try it out by opening another notebook, selecting the current Database and running this query in a new cell.

## Publish Cloud Function

After validating the Cloud Function interactively, you can publish it and use it as an API server for your data!

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>